In [1]:
### Importación de clases y paquetes
from pyspark.sql import functions as Funct
from HDFSContext import HDFSContext
from GenericDataFrame import GenericDataFrame
from ExceptionManager import ExceptionManager
from Dbscan import Dbscan
from pyspark.sql.functions import to_timestamp, col, regexp_replace
import datetime
from datetime import timedelta
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

 

In [2]:
#Método para limpiar los datos a nivel de mpa-reduce

def Limpiar(x):
    id = x[0]
    datos = list(x[1])
    salida = pd.DataFrame()
    ids = []
    fechas = []
    potencia = []
    calidad = []
    
    for i in range(0,len(datos)):
        ids.append(datos[i][0])
        fechas.append(datos[i][1])
        potencia.append(datos[i][2])
        calidad.append(datos[i][3])
    
    salida['Id'] = ids
    salida['Fecha'] = fechas
    salida['Potencia'] = potencia
    salida['Calidad'] = calidad
    
    otliers,datos_limpios = Limpiar_Datos(salida.set_index('Fecha'))
    
    return (id, [otliers,datos_limpios,salida])

def Limpiar_Datos(datosDestino):
    dbscan = Dbscan(datosDestino)
    dbscan.Limpiar_outliers()
    dbscan.Resumen_Datos()
    return dbscan.outliers,dbscan.datos_limpios

def Completar_Datos(x,fecha_inicio,fecha_fin):
    id = x[0]
    datos = list(x[1])
    
    outliers = datos[0].reset_index()
    datos_limpios = datos[1].reset_index()
    datos_originales = datos[2]
    
    
    if(datos_limpios['Id'].count()==0):
        datos_limpios = outliers
    
    resultado = pd.merge(datos_originales,datos_limpios,on='Id')
    resultado = resultado[resultado['Calidad'].isin(['Normal','AL','AL.L5','AL.L6','L1'])]
    
    if(resultado['Id'].count()==0):
        resultado = pd.merge(datos_originales,datos_limpios,on='Id')
    
    
    puntoInicial = np.array([datetime.datetime(fecha_inicio.year,fecha_inicio.month,fecha_inicio.day,0,0,0) - timedelta(seconds=1)])
    puntoInicial = pd.DataFrame(puntoInicial,columns=['Fecha'])
    
    puntoFinal = np.array([datetime.datetime(fecha_inicio.year,fecha_inicio.month,fecha_inicio.day,23,59,59) + timedelta(seconds=1)])
    puntoFinal = pd.DataFrame(puntoFinal,columns=['Fecha'])

    resultado = resultado[['Id','Fecha_x','Potencia_x','Calidad']]\
    .rename(columns={'Fecha_x':'Fecha','Potencia_x':'Potencia'})
    
    maximo = resultado['Id'].values[-1]
    
    resultado = pd.merge(puntoInicial,resultado,on='Fecha',how='outer')
    resultado = pd.merge(puntoFinal,resultado,on='Fecha',how='outer')

    values = {'Fecha':datetime.datetime(1970,1,1,0,0,0),'Id':maximo,'Potencia':0,'Calidad':'Normal'}
    resultado = resultado.fillna(value=values)
    resultado = resultado.set_index('Fecha').resample('S').mean()
    resultado = resultado[1:-1]
    resultado = resultado.interpolate(method='values')
    resultado = resultado.interpolate(method='values',limit_direction='backward')
    
    resultado_segundo = resultado.copy()
    
    resultado = resultado.resample('T').max()
    resultado['Elemento'] = id

   # if(tipo=='Origen'):
    #    calidad_circuitos = Obtener_Calidad_Circuitos(fecha_inicio,fecha_fin,circuito=id,destino=False)
    #else:
    #    calidad_circuitos = Obtener_Calidad_Circuitos(fecha_inicio,fecha_fin,circuito=id,destino=True)
        
    #calidad_circuitos = calidad_circuitos.toPandas().set_index('Fecha')[['Calidad','LimMaxOperacion',
    #                                                                     'LimOperacionContinuo','LimTermico',
    #                                                                     'TagCalidad','TagPotencia']]
    
    data_extract = ElementosTransmisionBL()
    extract_data_tranformador = data_extract.Extract_data_Transformador(fecha_inicio, fecha_fin,id)
        
    calidad_trafo = extract_data_tranformador.where(extract_data_tranformador.CodigoTrafo == id)
    
    calidad_trafo = calidad_trafo.toPandas().set_index('Fecha')[['Calidad','MVA_Emergente',
                                                                         'MVA_NominalFOA','Subestacion',
                                                                         'UnidadNegocio','CodigoTrafo']]
    
    
    resultado = pd.merge(calidad_trafo,resultado,on='Fecha')

    resultado_segundo = resultado_segundo.reset_index().groupby('Potencia')['Fecha'].min().reset_index()\
    .set_index('Potencia').rename(columns={'Fecha':'FechaMaximo'})
    resultado = resultado.reset_index().set_index('Potencia')
    resultado = pd.merge(resultado_segundo,resultado,on='Potencia')
    resultado = resultado.reset_index().set_index('Fecha').sort_index().reset_index()
    
    return (id,resultado)

In [3]:
class ElementosTransmisionBL():
    """Clase que procesa la lógica del negocio y limpieza de datos de los elementos de Transmisión."""
    
    def __init__ (self):
        self._genericDataFrame=GenericDataFrame(HDFSContext(Path='PI_INTEGRATOR',DataBase='TRANSMISION',Schema='')) 
        
    def Extract_data_Transformador(self, fecha_inicio,fecha_fin,trafo):
        """Método que realiza la extracción de datos de transformadores del HDFS"""
        
        try: 
            df_transformadores = self._genericDataFrame.GetDataCsvHdfs(tableName='',fileName='transformadores_*',header='true',
                                                                       delimiter='\t')
            
            df_transformadores = df_transformadores.select('Id', 
                                               to_timestamp(col('Estampa de Tiempo'),'dd/MM/yy HH:mm:ss').alias('Fecha'),
                                               col('Calidad Potencia').alias('Calidad'),
                                               regexp_replace(col('MVA_Emergente'),',','.').cast('double').alias('MVA_Emergente'),
                                               regexp_replace(col('MVA_NominalFOA'),',','.').cast('double').alias('MVA_NominalFOA'),
                                               'Subestacion','UnidadNegocio',
                                               col('Codigo').alias('CodigoTrafo'),
                                               col('Nombre').alias('NombreTrafo'), 
                                               regexp_replace(col('Potencia Aparente'),',','.').cast('double').alias('Potencia'))
            
            if trafo is None: 
                trafos = df_transformadores.select('*').where(df_transformadores.Fecha.between(fecha_inicio, fecha_fin))
            else: 
                trafos = df_transformadores.select('*').where((df_transformadores.CodigoTrafo == trafo)
                                                            & (df_transformadores.Fecha.between(fecha_inicio, fecha_fin)))
                
            return trafos
    
        except Exception as error:
            ExceptionManager.Treatment(error)
            
    def Transform_data_Transformador(self, fecha_inicio,fecha_fin,trafo): 
        """Método que realiza la transformación de datos de los transformadores. Se aplica el algoritmo DBCSAN para limpiar y 
        eliminar datos atipicos. """
        
        try: 
            
            #Procesamiento de cada transformador 
            
            data_extract = ElementosTransmisionBL()
            extract_data_tranformador = data_extract.Extract_data_Transformador(fecha_inicio, fecha_fin,trafo)
            
            transformador_map = extract_data_tranformador.rdd.map(lambda x: (x.CodigoTrafo,[x.Id,x.Fecha,x.Potencia, x.Calidad]))
            transformador_map_groupBy = transformador_map.groupByKey()
            transformador_map_clear = transformador_map_groupBy.map(lambda x: Limpiar(x))
            transformador_map_complete = transformador_map_clear.map(lambda x: Completar_Datos(x,fecha_inicio,fecha_fin))
            
            print(transformador_map_complete.collect())

        except Exception as error:
            ExceptionManager.Treatment(error)
    
    def Elt_main(self, fecha_inicio, fecha_fin,trafo=None):
        """Método que procesa todo el flujo de proceso del ETL."""
        print("---- Proceso de ETL del Análisis de potencia de Transformadores ---- \n")
        print("DATAMART: dwh_sirio")
        print("TABLE DE HECHOS: fact_transformador \n")
        
        print("1. Extracción de datos")
        extract_data_tranformador = self.Extract_data_Transformador(fecha_inicio, fecha_fin,trafo)
        
        if(extract_data_tranformador.count()==0):
            print(' **** WARNING: No existe Datos de Transformadores para Procesar. '+str(datetime.datetime.now()))
            return 
            
        print("2. Transformación de datos")
        transform_data = self.Transform_data_Transformador(fecha_inicio,fecha_fin,trafo)
        
        #print("3. Cargar  datos\n")
        #self.Load_data(transform_data,"cen_dws.fact_tmp_descon_carga")
        #self.Load_data(transform_data_horas,"cen_dws.dim_hora")

In [4]:
ob = ElementosTransmisionBL()
fecha_inicio = datetime.datetime.strptime('2019-10-10 00:00', '%Y-%m-%d %H:%M')
fecha_fin = datetime.datetime.strptime('2019-10-10 23:59', '%Y-%m-%d %H:%M')

ob.Elt_main(fecha_inicio, fecha_fin,'AMB138069T1')

---- Proceso de ETL del Análisis de potencia de Transformadores ---- 

DATAMART: dwh_sirio
TABLE DE HECHOS: fact_transformador 

1. Extracción de datos
2. Transformación de datos
[('AMB138069T1',                  Fecha   Potencia         FechaMaximo Calidad  MVA_Emergente  \
0  2019-10-10 00:00:00  12.213549 2019-10-10 00:00:00  Normal           48.1   
1  2019-10-10 01:00:00  11.494312 2019-10-10 00:48:53  Normal           48.1   
2  2019-10-10 02:00:00  11.115995 2019-10-10 01:38:23  Normal           48.1   
3  2019-10-10 03:00:00  10.970223 2019-10-10 02:10:33  Normal           48.1   
4  2019-10-10 04:00:00  11.119410 2019-10-10 02:23:13  Normal           48.1   
5  2019-10-10 05:00:00  11.358024 2019-10-10 04:59:53  Normal           48.1   
6  2019-10-10 06:00:00  10.451732 2019-10-10 05:59:23  Normal           48.1   
7  2019-10-10 07:00:00  10.431569 2019-10-10 07:00:03  Normal           48.1   
8  2019-10-10 08:00:00  11.840643 2019-10-10 07:56:18  Normal           48.1   
9  2